<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/parity_paradigm_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pennylane numpy scikit-learn umap-learn matplotlib

In [ ]:
#!/usr/bin/env python3
import itertools
import numpy as np
import pennylane as qml
from pennylane import numpy as pnp
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# 1) Data generation
def generate_parity_data(n_bits=4):
    X = np.array(list(itertools.product([0, 1], repeat=n_bits)))
    y = X.sum(axis=1) % 2
    return X, y

# 2) Exact analytic CNOT cascade
def analytic_cnot_cascade(X, n_bits=4):
    dev = qml.device("default.qubit", wires=n_bits + 1)

    @qml.qnode(dev)
    def circuit(x):
        for i, bit in enumerate(x):
            if bit:
                qml.PauliX(wires=i)
        for i in range(n_bits):
            qml.CNOT(wires=[i, n_bits])
        return qml.expval(qml.PauliZ(wires=n_bits))

    preds = []
    for x in X:
        exp = circuit(x)
        preds.append(0 if exp > 0 else 1)
    return np.array(preds)

# 3) Variational QNN with 5-fold CV
def variational_qnn_cv(X, y, n_bits=4, epochs=30, lr=0.4):
    dev = qml.device("default.qubit", wires=n_bits)

    @qml.qnode(dev, interface="autograd")
    def circuit(params, x):
        for i, bit in enumerate(x):
            if bit:
                qml.RY(np.pi, wires=i)
        for i in range(n_bits):
            qml.RY(params[i], wires=i)
        for i in range(n_bits - 1):
            qml.CNOT(wires=[i, i + 1])
        return qml.expval(qml.PauliZ(wires=n_bits - 1))

    y_signed = 1 - 2 * y
    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    scores = []

    for train_idx, test_idx in kf.split(X, y):
        params = pnp.random.randn(n_bits, requires_grad=True) * 0.1
        opt = qml.GradientDescentOptimizer(stepsize=lr)

        for _ in range(epochs):
            def cost(p):
                preds = [circuit(p, X[i]) for i in train_idx]
                return pnp.mean((pnp.array(preds) - y_signed[train_idx]) ** 2)
            params = opt.step(cost, params)

        preds_test = [circuit(params, X[i]) for i in test_idx]
        bits = [0 if p > 0 else 1 for p in preds_test]
        scores.append(accuracy_score(y[test_idx], bits))

    return np.mean(scores), np.std(scores)

# 4) Improved quantum kernel + SVM with hyperparameter tuning
def improved_quantum_kernel_svm(X, y, n_bits=4, layers=2):
    # 4.1 Prepare a device and random feature-map weights
    dev = qml.device("default.qubit", wires=n_bits)
    rng = np.random.RandomState(0)
    feature_weights = rng.randn(layers, n_bits, 3)

    @qml.qnode(dev)
    def feature_map_state(x):
        qml.templates.AngleEmbedding(x, wires=range(n_bits))
        qml.templates.StronglyEntanglingLayers(feature_weights, wires=range(n_bits))
        return qml.state()

    # 4.2 Build kernel matrix K_ij = |⟨ψ(x_i)|ψ(x_j)⟩|^2
    N = len(X)
    states = [feature_map_state(x) for x in X]
    K = np.zeros((N, N))
    for i in range(N):
        for j in range(N):
            ov = np.vdot(states[i], states[j])
            K[i, j] = np.abs(ov) ** 2

    # 4.3 Compute kernel-target alignment
    y_s = 1 - 2 * y
    T = np.outer(y_s, y_s)
    normF = lambda M: np.linalg.norm(M, 'fro')
    alignment = (np.sum(K * T)) / (normF(K) * normF(T))

    # 4.4 Hyperparameter tuning for precomputed-kernel SVM
    param_grid = [
        {"C": 0.1, "class_weight": None},
        {"C": 0.1, "class_weight": "balanced"},
        {"C": 1.0, "class_weight": None},
        {"C": 1.0, "class_weight": "balanced"},
        {"C": 10.0, "class_weight": None},
        {"C": 10.0, "class_weight": "balanced"},
    ]
    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    best = {"mean_acc": 0}

    for params in param_grid:
        scores = []
        for train_idx, test_idx in kf.split(X, y):
            svc = SVC(kernel="precomputed", **params)
            K_train = K[np.ix_(train_idx, train_idx)]
            svc.fit(K_train, y[train_idx])
            K_test = K[np.ix_(test_idx, train_idx)]
            preds = svc.predict(K_test)
            scores.append(accuracy_score(y[test_idx], preds))
        mean_acc, std_acc = np.mean(scores), np.std(scores)
        if mean_acc > best["mean_acc"]:
            best = {
                "params": params,
                "mean_acc": mean_acc,
                "std_acc": std_acc
            }

    return alignment, best

if __name__ == "__main__":
    n_bits = 4
    X, y = generate_parity_data(n_bits)

    # Exact analytic
    preds_exact = analytic_cnot_cascade(X, n_bits)
    acc_exact = accuracy_score(y, preds_exact)
    print(f"Analytic CNOT Cascade Accuracy: {acc_exact:.3f}")

    # Variational QNN
    mean_var, std_var = variational_qnn_cv(X, y, n_bits)
    print(f"Variational QNN CV Accuracy: {mean_var:.3f} ± {std_var:.3f}")

    # Improved Quantum Kernel + SVM
    align, best = improved_quantum_kernel_svm(X, y, n_bits, layers=2)
    print(f"Quantum-Kernel–Target Alignment: {align:.3f}")
    print(f"Best SVM Hyperparameters: {best['params']}")
    print(f"Quantum Kernel+SVM CV Accuracy: {best['mean_acc']:.3f} ± {best['std_acc']:.3f}")